| Version | Published Date| Details |
| -- | -- | -- |
| ver.1.0.0 | 2023/8/29 | 初版 |

# 画像の操作を自動化しよう

デジタルカメラを持っている人や，スマートフォンからTwitterやFacebookに写真をアップロードする人は，デジタル画像ファイルと接する機械が頻繁にあるでしょう。Windowsのペイントのような基本的なグラフィックソフトや，Adobe Photoshopのような高度なアプリをご存知の方もいるかもしれません。しかし大量の画像を編集する必要があると，手作業で編集するのは時間がかかるうえに退屈です。

そこでPythonを使います。[Pillow](https://pillow.readthedocs.io/en/stable/) はサードパーティの画像処理モジュールで，切り抜き，サイズ変更，画像内容の編集を簡単に行なえます。ペイントやPhotoshopのようなソフトと同様に画像処理できる能力を駆使して，何百枚，何千枚もの画像を簡単に自動編集できます。

# コンピュータ画像の基礎

画像操作の説明の前に，コンピュータがどのように画像を表現しているのかを説明します。つまりコンピュータがどのように画像の色や座標を扱っているのか，またPillowではどのように色と座標を扱うのか，基本を説明します。

## 色とRGBA値

コンピュータは画像の色を **RGBA値** で表現します。RGBA値とは，色の三原色 (赤, 緑, 青) の度合いと **アルファ値** 不透明度をひとまとめにしたものです。Red, Green, Blue, AlphaでRGBAということですね。それぞれの値は0から255の値を取り， **ピクセル** (画素) のひとつずつにRGBA値が割り当てられます。ピクセルとは，コンピュータの画面で表示できる最小の単色の点のことであり，画面は何百万ものピクセルから構成されています。

ピクセルのRGB値は，表示する色の濃淡の値そのものを示します。アルファ値は不透明度を示します。背景画像やデスクトップの壁紙に重ねて画像を表示するとき，アルファ値はどれだけ背景が透き通らないかを示します。0ならば透明で背景が完全に透き通り，255なら不透明で背景は見えません。

Pillowでは，RGBA値は4つの整数値のタプルで表されます。たとえば，赤い色は `(255, 0, 0, 255)` で表されます。つまり，赤は最大値，緑と青の成分はなく，アルファ値が最大で不透明になります。同様に，緑は `(0, 255, 0, 255)` 青は `(0, 0, 255, 255)` です。白は全成分が最大値の `(255, 255, 255, 255)` 黒は色成分のない `(0, 0, 0, 255)` になります。アルファ値が0なら，RGBの成分に関係なく不可視になります。つまり，不可視の赤は不可視の黒と同じです。

PillowはHTMLが採用している標準色名を使います。以下が抜粋です。

|  色名  |  RGBA値  |
| ---- | ---- |
|  White  |  `(255, 255, 255, 255)`  |
|  Green  |  `(0, 128, 0, 255)`  |
|  Gray   |  `(128, 128, 128, 128)` |
|  Black  |  `(0, 0, 0, 255)` |
|  Red    |  `(255, 0, 0, 255)` |
|  Blue   |  `(0, 0, 255, 255)` |
|  Yellow |  `(255, 255, 0, 255)` |
|  Purple |  `(128, 0, 128, 255)` |

Pilloweには `ImageColor.getcolor()` 関数があり，色名に対応するRGBAを検索できるので，RGBA値を覚えておく必要はありません。この関数の第1引数に色名の文字列を，第2引数に `'RGBA'` という文字列を渡すと，RGBAのタプルを返します。


In [1]:
from PIL import ImageColor

In [ ]:
ImageColor.getcolor('red', 'RGBA')

In [ ]:
ImageColor.getcolor('RED', 'RGBA')

In [ ]:
ImageColor.getcolor('Black', 'RGBA')

In [ ]:
ImageColor.getcolor('chocolate', 'RGBA')

In [ ]:
ImageColor.getcolor('CornflowerBlue', 'RGBA')

まずPILから `ImageColor` モジュールをインポートします。PillowではなくPILであることに注意してください。 `ImageColor.getcolor()` に渡す色名は大文字と小文字を区別しないので `red` と `RED` は同じRGBAタプルを返します。 `chocolate` や `CornflowerBlue` といった少し珍しい色名も渡せます。

Pillowがサポートしている大量の色名に関しては [Wikipedia - ウェブカラー](https://ja.wikipedia.org/wiki/%E3%82%A6%E3%82%A7%E3%83%95%E3%82%99%E3%82%AB%E3%83%A9%E3%83%BC) を参考にしてみてください。

## 座標と矩形タプル

画像のピクセルの位置はXY座標，すなわち画像上の水平位置と垂直位置で表されます。ピクセルの **原点 (origin)** は画像の左上の点であり， `(0, 0)` で表されます。最初の0はX座標を表し，原点が0で右に行くほど増えていきます。2つ目の0はY座標を表し，原点が0で下に行くほど増えていきます。これはとても重要です。数学で習ったY座標と逆なので注意してください。

<img src="https://drive.google.com/uc?id=1qRelceGERcTv1U4AAmKEgU4Azrfr3hBJ" width="720px">

多くのPillowの関数やメソッドは **矩形タプル** の引数を取ります。つまり，画像中の矩形領域を表す4つの整数値のタプルを要求します。4つの整数は順番に次のとおりです。

- **左**: 矩形の左上点のX座標
- **上**: 矩形の左上点のY座標
- **右**: 矩形の右下点のX座標。左の値より大きくなければならない
- **下**: 矩形の右下点のY座標。上の値より大きくなければならない

矩形には，左上点の座標が含まれますが，右下点の座標は含まれないことに注意が必要です。たとえば `(3, 1, 9, 6)` というタプルは，以下の黒いピクセルの矩形領域を表します。

<img src="https://drive.google.com/uc?id=1jhNDGM_uQfBkrAJET2oD2q1aEfjATkq5" width="480px">

## Pillow で画像を操作する

色と座標の扱い方を学んだので，さっそくPillowを使って画像を操作してみましょう。まず準備としてGoogle Driveをマウントして画像を扱えるようにします。

In [7]:
import requests
from google.colab import drive
from PIL import Image
from io import BytesIO

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


続いて，今回のStation作業用ディレクトリを `python_station` という名前で作成します。 **このセルは一度だけ実行するようにしてください。** 2回以上実行すると「ファイルがすでに存在します」といったエラーが出ます。

In [8]:
'''
import os
mydrive_path = '/content/drive/MyDrive'
os.chdir(mydrive_path)
station_path = os.path.join(mydrive_path, 'python_station')
os.mkdir(station_path)
'''

"\nimport os\nmydrive_path = '/content/drive/MyDrive'\nos.chdir(mydrive_path)\nstation_path = os.path.join(mydrive_path, 'python_station')\nos.mkdir(station_path)\n"

In [9]:
import os

image_url = 'https://drive.google.com/uc?id=1UynZ7dYkNgPYcUcSntLur-kC8uFp-UJR'
response = requests.get(image_url)

image = Image.open(BytesIO(response.content))
mydrive_path = '/content/drive/MyDrive/python_station'
save_path = os.path.join(mydrive_path, 'python.png')

image.save(save_path)

上のセルを実行すると画像をGoogle Driveに保存できます。 `python.png` が保存できたら，読み込んでみましょう。かわいらしいヘビの画像が保存されましたか？

In [ ]:
python_im = Image.open('/content/drive/MyDrive/python_station/python.png')
python_im

もしこのStationを途中から再開したい場合は，作業用のディレクトリに移動する必要があります。その場合は以下のセルを実行して，ドライブのマウント→作業ディレクトリへの移動を行うようにしてください。

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
mydrive_path = '/content/drive/MyDrive/python_station'
os.chdir(mydrive_path)

画像を読み込むには，Pillowから `Image` モジュールをインポートし `Image.open()` にファイル名を渡して呼び出します。そして戻り地を `python_im` のような変数に格納します。Pillowのモジュール名は `PIL` です。これは Python Image Library という古いモジュールとの互換性のためです。したがって

```
from Pillow import Image
```

ではなく

```
from PIL import Image
```

のように書きます。Pillowの設計方針に従いこのように書くことが推奨されています。

`Image.open()` 関数は `Image` オブジェクトを返します。さまざまなフォーマットの画像ファイルを `Image.open()` 関数に渡すだけで `Image` オブジェクトとして読み込むことができます。 `Image` オブジェクトに変更を加えたら `save()` メソッドを使ってさまざまなフォーマットの画像ファイルに保存できます。回転やサイズ変更，切り抜き，描画といった画像操作は，すべて `Image` オブジェクトのメソッド呼び出しを通して行います。

以下の例では `python.png` を読み込んだ `Image` オブジェクトが変数 `python_im` に格納されているとします。

`Image` オブジェクトの `size` 属性には，画像の幅と高さをピクセル単位で表したタプルが格納されています。


In [ ]:
python_im.size

変数 `width` と `height` に代入すれば `width` や `height` を使ってアクセスできます。

In [15]:
width, height = python_im.size

In [ ]:
width

In [ ]:
height

`filename` 属性はファイル名を表します。

In [ ]:
python_im.filename

`format` と `format_description` 属性は，画像フォーマットを表す文字列です。

In [ ]:
python_im.format

`format_description` はより詳細な説明になっています。

In [ ]:
python_im.format_description

`save()` メソッドに `python.gif` を渡して呼び出すと `python.gif` というファイル名で新たに画像をGoogle Driveに保存します。Pillowはファイルの拡張子が `.gif` であるのを見て，自動的にGIF形式で画像を保存します。どちらも同じ画像に基づくものですが，画像形式が異なるため完全には一致しません。

In [21]:
python_im.save('python.gif')

PillowにはImageオブジェクトを返す `Image.new()` 関数もあります。`Image. open()` に似ていますが， `Image.new()` は空白の画像を返します。 `Image.new()` の引数は次のとおりです。

- 文字列 `'RGBA'` 。カラーモードをRGBAにする (ここでは扱わないカラーモードが他にもあります)
- サイズ。画像の幅と大きさを表す2つの整数のタプル
- 初期値として画像を塗りつぶす背景色。TGBA値を表す4つの整数のタプル。`ImageColor.getcolor()` 関数の戻り値を使ってもよいし，色名の文字列を直接渡してもよい

まず 幅100×高さ200 で紫の背景色の画像を表す `Image` オブジェクトを生成します。それから画像を `purple_image.png` という名前で保存します。

In [ ]:
from PIL import Image
im = Image.new('RGBA', (100, 200), 'purple')
im.save('purple_image.png')
im

続いて `(20, 20)` の大きさで背景色のない `Image` オブジェクトを生成します。背景色を省略すると，不可視の黒 `(0, 0, 0, 0)` がデフォルトの色として使われるので `im2` は透明背景になります。この 20×20 の透明な正方形画像を `transparent_image.png` という名前で保存します。

In [ ]:
im2 = Image.new('RGBA', (20, 20))
im2.save('transparent_image.png')
im2

## 画像を切り抜く

画像の **切り抜き** は，画像中の矩形領域を選択して，矩形外を削除する操作です。 `Image` オブジェクトの `crop()` メソッドは，矩形タプルを受け取り，切り抜いた画像を表す `Image` オブジェクトを返します。切り抜き操作は **破壊的** ではありません。すなわち，元の `Image` オブジェクトは変更されず `crop()` メソッドは新しい `Image` オブジェクトを返します。繰り返しになりますが，切り抜き領域を表す矩形タプルには，左上の点は含まれますが，右下の点は含まれません。

In [ ]:
from PIL import Image
python_im = Image.open('python.png')
cropped_im = python_im.crop((10, 30, 200, 150))
cropped_im

こうしてヘビの顔が切り出せました。これを `cropped_im` に格納し `save()` を呼び出して `cropped.png` に保存します。

In [25]:
cropped_im.save('cropped.png')

## 画像のコピー&ペースト

`copy()` メソッドは，呼び出された `Image` オブジェクトと同じ画像データを持つ新しい `Image` オブジェクトを生成して返します。画像に変更を加えたいが，元の変更しないバージョンも残しておきたい場合に便利です。

In [26]:
from PIL import Image
python_im = Image.open('python.png')
python_copy_im = python_im.copy()

変数 `python_im` と `python_copy_im` はそれぞれ別の `Image` オブジェクトであり， `python_copy_im` には `python_im` の画像ゼータの複製が格納されています。したがって `python_copy_im` を変更しても `python_im` には影響はありません。

それでは `python_copy_im` を `paste()` メソッドを使って変更してみます。

まず `crop()` に矩形タプルを渡して，ヘビの顔の部分に相当する `python.png` の矩形領域を切り抜きます。これで切り抜いた領域を表す `Image` オブジェクトが生成されるので `python_im` に格納します。

In [ ]:
python_im = Image.open('python.png')
face_im = python_im.crop((10, 30, 200, 150))
face_im.size

続いて `face_im` を `python_copy_im` に貼り付けます。 `paste()` メソッドは，貼り付ける `Image` オブジェクトとそれを貼り付ける左上点のXY座標を表すタプルの，2つの引数を取ります。ここでは `paste()` を3回呼び出して `face_im` を `python_copy_im` の `(0, 0,)` と `(100, 200)` と `(200, 300)` に貼り付けています。

In [28]:
python_copy_im.paste(face_im, (0, 0))
python_copy_im.paste(face_im, (100, 200))
python_copy_im.paste(face_im, (200, 300))

変更した `python_copy_im` はこのようになりました。

In [ ]:
python_copy_im

最後に変更した `python_copy_im` を `pasted.png` に保存します。

In [30]:
python_copy_im.save('pasted.png')

`paste()` メソッドは `Image` オブジェクトを「インプレース」で変更する点に注意してください。つまり貼り付けた画像を表す新しい `Image` オブジェクトを返すわけではないということです。 `paste()` を呼び出しつつ元の画像を変更したくない場合は `copy()` で画像の複製を作ったあと，複製に対して `paste()` を呼び出すようにします。

In [ ]:
python_im_width, python_im_height = python_im.size
face_im_width, face_im_height = face_im.size
python_copy_two = python_im.copy()

for left in range(0, python_im_width, face_im_width):
    for top in range(0, python_im_height, face_im_height):
        print(left, top)
        python_copy_two.paste(face_im, (left, top))

In [ ]:
python_copy_two

In [33]:
python_copy_two.save('tiled.png')

ここではまず `python_im` の幅と高さを `python_im_width` と `python_im_height` に格納します。そして `python_im` の複製を作って `python_im_two` に格納します。貼り付ける先の画像の複製ができたので `face_im` を `python_copy_two` に貼り付けるためにループを回します。外側の `for` ループでは，変数 `left` を0から `face_im_width(=120)` ずつ増やします。内側の `for` ループでは，変数 `top` を0から `face_im_height(=215)` ずつ増やします。
これらの入れ子になった `for` ループによって `left` と `top` の値を順番に変えながら， `python_copy_two` の上に `face_im` を敷き詰めるように貼り付けていきます。入れ子のループの様子を調べるために `left` と `top` の値を表示しています。貼り付けが完了したら，変更後の `python_copy_two` を `tiled.png` に保存します。

## 画像サイズを変更する

`resize()` メソッドを呼び出すと，指定した幅と高さにサイズを変更した新しい `Image` オブジェクトを返します。引数は新しい幅と高さを表す2つの整数のタプルです。

In [34]:
from PIL import Image
python_im = Image.open('python.png')
width, height = python_im.size

まず `python_im.size` のタプルを変数 `width` と `height` に代入します。 `python_im.size[0]` や `python_im.size[1]` と書いても動きますが `width` と `height` を使うほうがコードが読みやすくなります。

In [ ]:
quartersized_im = python_im.resize((width // 2, height // 2))
quartersized_im

まず新たな幅に `width // 2` 高さに `height // 2` を渡しているので `resize()` の返す `Image` オブジェクトは，元の画像の半分の幅と高さ，つまり1/4のサイズになります。 `resize()` の引数は整数のタプルのみ受け付けます。そのため `//` を使って整数の割り算をします。

In [ ]:
quartersized_im.save('quartersized.png')
svelte_im = python_im.resize((width, height + 300))
svelte_im

このサイズ変更では，幅と高さが同じ比率を保っていましたが，元の画像の縦横比と異なる幅と高さを指定しても構いません。変数 `svelte_im` に格納される画像は，幅は元の画像と同じですが，高さは300ピクセル高いので，被写体の蛇が細くなります。
`resize()` メソッドは元の `Image` オブジェクトを変更せず，新しい `Image` オブジェクトを返します。つまり，インプレースでないメソッドです。

画像の縦横比を維持したまま，あるサイズに収まるように縮小したい場合は `thumbnail()` メソッドを使うのが便利です。引数は最大の幅と高さを表す2つの整数のタプルで，画像が指定サイズを上回る場合には，ちょうど収まるように縮小してくれます。

In [ ]:
thumb_im = python_im.copy()
thumb_im.size

In [ ]:
thumb_im.thumbnail((100, 100))
thumb_im.size

In [ ]:
thumb_im

In [40]:
thumb_im.save('thumbnail.png')

`thumbnail()` メソッドはインプレースなメソッドであり，元画像が変更されることに注意しましょう。元画像を変更したいなら，上の例のように `copy()` を使って元画像を保存しておきます。

`thumbnail()` の戻り値は `None` なので，間違えて次のように書くと画像を失ってしまいます。

In [ ]:
# 間違っている例
thumb_im = thumb_im.thumbnail((100, 100))
thumb_im == None

## 画像を回転・反転する

`rotate()` メソッドを用いると，元の画像はそのままで，回転した画像を `Image` として返します。`rotate()` の引数は，反時計回りの回転角度を表す整数または浮動小数点数です。

In [42]:
from PIL import Image
python_im = Image.open('python.png')

In [43]:
python_im.rotate(90).save('rotated90.png')
python_im.rotate(180).save('rotated180.png')
python_im.rotate(270).save('rotated180.png')

In [ ]:
python_im.rotate(90)

In [ ]:
python_im.rotate(180)

In [ ]:
python_im.rotate(270)

ここでは `rotate()` が返す `Image` オブジェクトに対して直接 `save()` を呼び出す，メソッドの **連鎖 (Chain)** 呼び出しをしています。1行目は90度回転した画像を `rotate90.png` という名前で保存し，同様に2行目は180度，3行目は270度回転しています。

90度と270度の回転では，画像の幅と高さが変わる点に注意しましょう。それ以外の角度では元画像の幅と高さが維持されます。回転によって生じたすき間には透明なピクセルが埋められます。

In [ ]:
python_im.rotate(60).save('rotate60.png')
python_im.rotate(60)

この呼び出しでは，画像を60度回転させて `rotate60.png` に保存しています。この場合は回転した画像が収まるようにサイズが調整されています。

In [ ]:
python_im.rotate(60, expand=True).save('rotate60_expand.png')
python_im.rotate(60, expand=True)

`rotate()` メソッドにキーワード引数 `expand=True` を与えると，回転した画像が収まるように，新しい画像の幅や高さは大きくなります。

`transpose()` メソッドを用いると，画像を胸像反転できます。引数には `Image.Transpose.FLIP_LEFT_RIGHT` か `Image.Transpose.FLIP_TOP_BOTTOM` のどちらかを渡します。

In [ ]:
h_im = python_im.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
h_im

In [ ]:
v_im = python_im.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
v_im

In [51]:
h_im.save('horizontal_flip.png')
v_im.save('vertical_flip.png')

`rotate()` と同様に `transpose()` も新たに `Image` オブジェクトを生成するため，元の画像は変更されません。前半では `Image.Transpose.FLIP_LEFT_RIGHT` を渡して水平方向に反転した画像を `horizontal_flip.png` に保存しています。後半では `Image.Transpose.FLIP_TOP_BOTTOM` を渡して垂直方向に反転し `vertical_flip.png` に保存しています。

# 確認テスト

以下のコードの穴埋めをして，このStationで作った画像すべてを `100*100` のサムネイルにし，1行4枚で貼り付けた画像を作成してください。

In [ ]:
from PIL import Image
import os
import math

file_names = [name for name in os.listdir('.') if name.endswith(('.png', '.jpg', '.jpeg'))]
file_length = len(file_names)

#一行に４枚（縦横400,区画100*100）
white = Image.new('RGBA', (400, math.ceil(file_length / 4) * 100), 'white')

for idx, name in enumerate(file_names):
    # 画像を開く
    image = Image.open(name)
    # 画像をサムネイルにする
    image.thumbnail((100,100))
    #iは行番号、jは列番号(divmodで商と余り)
    i, j = divmod(idx, 4)
    left, right = i * 100, j * 100 # 画像の貼り付け位置を取得
    # 画像を貼り付ける
    white.paste(image, (left, right))

white